<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/Midas2Onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/isl-org/MiDaS.git
%cd MiDaS
!pip install torch torchvision
!pip install timm
!pip install einops
!pip install onnx


Cloning into 'MiDaS'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 622 (delta 183), reused 139 (delta 137), pack-reused 375 (from 1)
Receiving objects: 100% (622/622), 3.44 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (246/246), done.
/content/MiDaS
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.8 MB/s eta 0:00:00


In [2]:
import cv2
import torch
import utils
from midas.dpt_depth import DPTDepthModel
from midas.midas_net_custom import MidasNet_small
from midas.midas_net import MidasNet
import os
import requests
import gc

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [9]:
# Custom normalization layer
class NormalizationLayer(torch.nn.Module):
    def __init__(self, mean, std):
        super(NormalizationLayer, self).__init__()
        self.mean = mean.view(1, -1, 1, 1)
        self.std = std.view(1, -1, 1, 1)

    def forward(self, x):
        return (x - self.mean) / self.std


class StandardizationLayer(torch.nn.Module):
    def __init__(self):
        super(StandardizationLayer, self).__init__()

    def forward(self, x):
        # Use torch.min and torch.max instead of torch(x).min and torch(x).max
        return (x - torch.min(x)) / (torch.max(x) - torch.min(x))
# convert all models


onnxFile = "weights/" +" midas_v21_small_256" + ".onnx"


model_path = "/content/midas_v21_small_256.pt"
device = torch.device("cpu")


model = MidasNet_small(
    model_path,
    features=64,
    backbone="efficientnet_lite3",
    exportable=True,
    non_negative=True,
    blocks={'expand': True}
)


# specify input size

net_w, net_h = 256, 256



mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

# insert normalization layer at the beginning of the model
norm_layer = NormalizationLayer(mean, std)
stn_layer=StandardizationLayer()
model = torch.nn.Sequential(norm_layer, model,stn_layer)


model.eval()



Loading weights:  /content/midas_v21_small_256.pt


Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
/content/MiDaS/midas/base_model.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pa

Sequential(
  (0): NormalizationLayer()
  (1): MidasNet_small(
    (pretrained): Module(
      (layer1): Sequential(
        (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU6(inplace=True)
            (se): Identity()
            (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act2): Identity()
          )
        )
        (4): Sequential(
          (0): InvertedResidual(
            (conv_pw): Conv2d(24, 

In [11]:
output=model(torch.rand(1, 3, net_h, net_w, dtype=torch.float))
torch.min(output)

tensor(0., grad_fn=<MinBackward1>)

In [12]:
torch.onnx.export(
    model,
    torch.rand(1, 3, net_h, net_w, dtype=torch.float),
    onnxFile,
    export_params=True,
    opset_version=15,
    input_names=["input_image"],
    output_names=["output_depth"],
    do_constant_folding=True
)

# free memory
del model
gc.collect()

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:663: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:1186: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_typ

53